<a href="https://colab.research.google.com/github/leyli16/HousingPricePrediction/blob/main/housing_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# Data Wrangling and Cleaning

## Loading data through kaggle setup

In [2]:
%%capture
!pip instsall kaggle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

## Loading in USA Real Estate Dataset


In [5]:
#!/bin/bash
!!kaggle datasets download ahmedshahriarsakib/usa-real-estate-dataset

['Dataset URL: https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset',
 'License(s): other']

In [6]:
!unzip usa-real-estate-dataset.zip

Archive:  usa-real-estate-dataset.zip
  inflating: realtor-data.zip.csv    


In [8]:
df_raw = pd.read_csv('realtor-data.zip.csv')
df_raw.head(10)

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN
5,103378.0,for_sale,179000.0,4.0,3.0,0.46,1850806.0,San Sebastian,Puerto Rico,612.0,2520.0,NaN
6,1205.0,for_sale,50000.0,3.0,1.0,0.20,1298094.0,Ciales,Puerto Rico,639.0,2040.0,NaN
7,50739.0,for_sale,71600.0,3.0,2.0,0.08,1048466.0,Ponce,Puerto Rico,731.0,1050.0,NaN
8,81909.0,for_sale,100000.0,2.0,1.0,0.09,734904.0,Ponce,Puerto Rico,730.0,1092.0,NaN
9,65672.0,for_sale,300000.0,5.0,3.0,7.46,1946226.0,Las Marias,Puerto Rico,670.0,5403.0,NaN


In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zip_code        float64
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(8), object(4)
memory usage: 203.8+ MB


In [10]:
df_raw.describe()

,brokered_by,price,bed,bath,acre_lot,street,zip_code,house_size
count,2.221849e+06,2.224841e+06,1.745065e+06,1.714611e+06,1.900793e+06,2.215516e+06,2.226083e+06,1.657898e+06
mean,5.293989e+04,5.241955e+05,3.275841e+00,2.496440e+00,1.522303e+01,1.012325e+06,5.218668e+04,2.714471e+03
std,3.064275e+04,2.138893e+06,1.567274e+00,1.652573e+00,7.628238e+02,5.837635e+05,2.895408e+04,8.081635e+05
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
25%,2.386100e+04,1.650000e+05,3.000000e+00,2.000000e+00,1.500000e-01,5.063128e+05,2.961700e+04,1.300000e+03
50%,5.288400e+04,3.250000e+05,3.000000e+00,2.000000e+00,2.600000e-01,1.012766e+06,4.838200e+04,1.760000e+03
75%,7.918300e+04,5.500000e+05,4.000000e+00,3.000000e+00,9.800000e-01,1.521173e+06,7.807000e+04,2.413000e+03
max,1.101420e+05,2.147484e+09,4.730000e+02,8.300000e+02,1.000000e+05,2.001357e+06,9.999900e+04,1.040400e+09


In [11]:
df_raw.describe(include = 'object')

,status,city,state,prev_sold_date
count,2226382,2224975,2226374,1492085
unique,3,20098,55,14954
top,for_sale,Houston,Florida,2022-03-31
freq,1389306,23862,249432,17171


## Cleaning up USA Real Estate data

In [12]:
df_raw.duplicated().sum()

np.int64(0)

In [13]:
# Keep only the necessary columns
df_req_cols = df_raw [['price', 'bed', 'bath', 'acre_lot', 'city', 'state', 'zip_code', 'house_size', 'status']]

In [14]:
total_missing = df_req_cols.isna().sum()*100/len(df_req_cols)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
price,0.069215
bed,21.618797
bath,22.986666
acre_lot,14.624130
city,0.063197
state,0.000359
zip_code,0.013430
house_size,25.533983
status,0.000000


In [15]:
# Drop nulls, reset and drop the index
df_nadropped = df_req_cols.dropna().reset_index(drop=True)
df_nadropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360347 entries, 0 to 1360346
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   price       1360347 non-null  float64
 1   bed         1360347 non-null  float64
 2   bath        1360347 non-null  float64
 3   acre_lot    1360347 non-null  float64
 4   city        1360347 non-null  object 
 5   state       1360347 non-null  object 
 6   zip_code    1360347 non-null  float64
 7   house_size  1360347 non-null  float64
 8   status      1360347 non-null  object 
dtypes: float64(6), object(3)
memory usage: 93.4+ MB


In [16]:
total_missing = df_nadropped.isna().sum()*100/len(df_nadropped)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
price,0.0
bed,0.0
bath,0.0
acre_lot,0.0
city,0.0
state,0.0
zip_code,0.0
house_size,0.0
status,0.0


In [17]:
# cast date types
df_typed = df_nadropped.astype({'price': 'float', 'bed': 'int', 'bath': 'int', 'acre_lot': 'float', 'city':'str', 'state':'str', 'zip_code':'int', 'house_size': 'float'})

In [18]:
df_typed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360347 entries, 0 to 1360346
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   price       1360347 non-null  float64
 1   bed         1360347 non-null  int64  
 2   bath        1360347 non-null  int64  
 3   acre_lot    1360347 non-null  float64
 4   city        1360347 non-null  object 
 5   state       1360347 non-null  object 
 6   zip_code    1360347 non-null  int64  
 7   house_size  1360347 non-null  float64
 8   status      1360347 non-null  object 
dtypes: float64(3), int64(3), object(3)
memory usage: 93.4+ MB


In [19]:
# Filter rows with status = sold
df_sold = df_typed[df_typed['status'] == 'sold'].drop(columns=['status'])

In [20]:
# Adding a column price_per_sqft
df_sold['price_per_sqft'] = df_sold['price'] / df_sold['house_size']

In [21]:
df_sold.head()

,price,bed,bath,acre_lot,city,state,zip_code,house_size,price_per_sqft
750602,524900.0,3,2,0.09,Aguada,Puerto Rico,602,2200.0,238.590909
750603,90000.0,3,2,0.08,Aguadilla,Puerto Rico,603,1421.0,63.335679
750604,22500.0,2,1,0.32,Anasco,Puerto Rico,610,850.0,26.470588
750605,168000.0,6,4,0.05,Anasco,Puerto Rico,610,3422.0,49.094097
750606,200000.0,3,1,4.98,Arecibo,Puerto Rico,612,580.0,344.827586


In [22]:
df_real_estate = df_sold

## Loading in Zip code to County Dataset

In [23]:
#!/bin/bash
!!kaggle datasets download danofer/zipcodes-county-fips-crosswalk

['Dataset URL: https://www.kaggle.com/datasets/danofer/zipcodes-county-fips-crosswalk',
 'License(s): CC0-1.0']

In [24]:
!unzip zipcodes-county-fips-crosswalk.zip

Archive:  zipcodes-county-fips-crosswalk.zip
  inflating: ZIP-COUNTY-FIPS_2017-06.csv  


In [25]:
zip_county_df = pd.read_csv('ZIP-COUNTY-FIPS_2017-06.csv')
zip_county_df.head(10)

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1
5,36701,Autauga County,AL,1001,H1
6,36091,Autauga County,AL,1001,H1
7,36051,Autauga County,AL,1001,H1
8,36068,Autauga County,AL,1001,H1
9,36008,Autauga County,AL,1001,H1


In [26]:
zip_county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52889 entries, 0 to 52888
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ZIP         52889 non-null  int64 
 1   COUNTYNAME  52889 non-null  object
 2   STATE       52889 non-null  object
 3   STCOUNTYFP  52889 non-null  int64 
 4   CLASSFP     52889 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


## Cleaning up Zip Code to County Dataset

In [27]:
zip_county_df.duplicated().sum()

np.int64(0)

In [28]:
total_missing = zip_county_df.isna().sum() * 100 / len(zip_county_df)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
ZIP,0.0
COUNTYNAME,0.0
STATE,0.0
STCOUNTYFP,0.0
CLASSFP,0.0


In [29]:
# cast date types
zip_county_df = zip_county_df.astype({'ZIP': 'int', 'COUNTYNAME':'str', 'STATE':'str'})

In [30]:
zip_county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52889 entries, 0 to 52888
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ZIP         52889 non-null  int64 
 1   COUNTYNAME  52889 non-null  object
 2   STATE       52889 non-null  object
 3   STCOUNTYFP  52889 non-null  int64 
 4   CLASSFP     52889 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


In [31]:
zip_county_df.rename(columns={'ZIP': 'zip_code', 'COUNTYNAME':'county', 'STATE':'state'}, inplace=True)

In [32]:
zip_county_df.drop(columns=['STCOUNTYFP', 'CLASSFP'], inplace=True)

In [33]:
# count unique counties per ZIP code
zip_counts = zip_county_df['zip_code'].value_counts()

# Step 1: Find ZIP codes with only one county
zips_one_county = zip_counts[zip_counts == 1].index
# Step 2: Filter out rows with those ZIP codes
zip_county_df = zip_county_df[zip_county_df['zip_code'].isin(zips_one_county)]

In [34]:
zip_county_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29102 entries, 0 to 52888
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   zip_code  29102 non-null  int64 
 1   county    29102 non-null  object
 2   state     29102 non-null  object
dtypes: int64(1), object(2)
memory usage: 909.4+ KB


## Create new column "county" in USA Real Estate Dataset

In [35]:
# Merge df with zip_county_df to get county info
real_estate_df = pd.merge(df_real_estate, zip_county_df, on='zip_code', how='inner')
real_estate_df.drop(columns=['state_x'], inplace=True)
real_estate_df.rename(columns={'state_y':'state'},inplace=True)
real_estate_df.head(10)

,price,bed,bath,acre_lot,city,zip_code,house_size,price_per_sqft,county,state
0,22500.0,2,1,0.32,Anasco,610,850.0,26.470588,Anasco Municipio,PR
1,168000.0,6,4,0.05,Anasco,610,3422.0,49.094097,Anasco Municipio,PR
2,120000.0,4,2,0.18,Sabana Grande,637,1188.0,101.010101,Sabana Grande Municipio,PR
3,133000.0,3,1,0.60,Hatillo,659,1049.0,126.787417,Hatillo Municipio,PR
4,177000.0,2,2,0.10,Isabela,662,4674.0,37.869063,Isabela Municipio,PR
5,220000.0,5,3,0.11,Isabela,662,2992.0,73.529412,Isabela Municipio,PR
6,175000.0,3,3,0.45,Las Marias,670,3090.0,56.634304,Las Marias Municipio,PR
7,149000.0,3,2,0.28,Mayaguez,682,2000.0,74.500000,Mayaguez Municipio,PR
8,2750000.0,3,4,0.15,San Juan,907,2820.0,975.177305,San Juan Municipio,PR
9,185000.0,3,2,13.47,Maunabo,707,1159.0,159.620362,Maunabo Municipio,PR


In [36]:
real_estate_df.shape

(451948, 10)

In [37]:
real_estate_df.describe()

,price,bed,bath,acre_lot,zip_code,house_size,price_per_sqft
count,4.519480e+05,451948.000000,451948.000000,451948.000000,451948.000000,451948.000000,451948.000000
mean,5.518665e+05,3.284531,2.428421,11.169715,59467.386436,1943.216633,285.079374
std,7.508103e+05,1.316137,1.195875,774.451696,31666.360915,1763.082967,240.109022
min,1.000000e+00,1.000000,1.000000,0.000000,610.000000,100.000000,0.000282
25%,2.499075e+05,3.000000,2.000000,0.130000,31312.000000,1303.000000,150.130548
50%,3.950000e+05,3.000000,2.000000,0.190000,70433.000000,1714.000000,214.868181
75%,6.250000e+05,4.000000,3.000000,0.320000,91106.000000,2300.000000,333.266667
max,8.000000e+07,444.000000,222.000000,100000.000000,99402.000000,927828.000000,15015.015015


In [38]:
real_estate_df.describe(include = 'object')

,city,county,state
count,451948,451948,451948
unique,8090,1267,51
top,Houston,Los Angeles County,CA
freq,10570,20836,93327


## Loading in Zip Code Demographics

In [39]:
#!/bin/bash
!!kaggle datasets download erdi28/zip-codes-demographics

['Dataset URL: https://www.kaggle.com/datasets/erdi28/zip-codes-demographics',
 'License(s): MIT']

In [40]:
!unzip zip-codes-demographics.zip

Archive:  zip-codes-demographics.zip
  inflating: zip_code_demographics.csv  


In [41]:
zip_code_demographics_df_raw = pd.read_csv('zip_code_demographics.csv')
zip_code_demographics_df_raw.head(10)

,zip,lat,lng,city,state_id,state_name,population,density,county_name,po_box,dist_highway,dist2_large_airport,dist2_medium_airport,dist_to_shore,number_of_business,adjusted_gross_income,total_income_amount,number_of_returns
0,1001,42.06262,-72.62521,Agawam,MA,Massachusetts,16088,550.1,Hampden,0,1.387035,106.145765,12.946212,93.049251,438.0,598807,6.047690e+05,9320
1,1002,42.37633,-72.46462,Amherst,MA,Massachusetts,27323,198.1,Hampshire,0,14.438177,112.264368,21.080079,133.370144,571.0,989558,1.005796e+06,9880
2,1005,42.42117,-72.10655,Barre,MA,Massachusetts,4947,44.2,Worcester,0,16.788339,90.664964,25.547718,97.639881,97.0,164207,1.660540e+05,2490
3,1007,42.28163,-72.40009,Belchertown,MA,Massachusetts,15304,107.7,Hampshire,0,13.663839,101.552921,14.762395,114.406034,217.0,647074,6.547390e+05,7970
4,1008,42.18234,-72.95819,Blandford,MA,Massachusetts,1171,7.4,Hampden,0,2.593655,136.548797,20.177950,107.466779,18.0,47826,4.824100e+04,660
5,1010,42.12904,-72.20597,Brimfield,MA,Massachusetts,3703,40.6,Hampden,0,4.737271,78.645500,28.049262,94.928024,73.0,155666,1.576770e+05,1980
6,1011,42.30233,-72.96448,Chester,MA,Massachusetts,1332,15.7,Hampden,0,8.058693,142.414627,26.041908,128.563170,13.0,38223,3.855300e+04,630
7,1012,42.38495,-72.84675,Chesterfield,MA,Massachusetts,503,16.1,Hampshire,0,18.331096,138.381684,27.466664,137.693285,10.0,24826,2.510200e+04,410
8,1013,42.16059,-72.60788,Chicopee,MA,Massachusetts,22709,1549.1,Hampden,0,0.062286,109.114246,7.075893,104.576258,317.0,516431,5.201740e+05,11270
9,1020,42.17618,-72.56538,Chicopee,MA,Massachusetts,30704,951.4,Hampden,0,0.925140,106.769095,3.205533,105.789421,550.0,828125,8.346140e+05,15760


In [42]:
zip_code_demographics_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33704 entries, 0 to 33703
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   zip                    33704 non-null  int64  
 1   lat                    33704 non-null  float64
 2   lng                    33704 non-null  float64
 3   city                   33704 non-null  object 
 4   state_id               33704 non-null  object 
 5   state_name             33704 non-null  object 
 6   population             33704 non-null  int64  
 7   density                33704 non-null  float64
 8   county_name            33704 non-null  object 
 9   po_box                 33704 non-null  int64  
 10  dist_highway           33704 non-null  float64
 11  dist2_large_airport    33704 non-null  float64
 12  dist2_medium_airport   33704 non-null  float64
 13  dist_to_shore          33704 non-null  float64
 14  number_of_business     33624 non-null  float64
 15  ad

In [43]:
zip_code_demographics_df_raw.describe()

,zip,lat,lng,population,density,po_box,dist_highway,dist2_large_airport,dist2_medium_airport,dist_to_shore,number_of_business,adjusted_gross_income,total_income_amount,number_of_returns
count,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33704.000000,33624.000000,3.370400e+04,3.370400e+04,33704.000000
mean,49842.607821,38.888195,-91.051366,11287.959738,528.704068,0.179949,29.291294,158.648048,40.854624,304.798736,260.694920,4.251341e+05,4.298943e+05,5341.873665
std,27451.111442,5.228075,15.460095,15200.216997,2219.277025,0.384151,174.104869,166.207042,80.224956,298.948686,432.112814,7.561519e+05,7.644416e+05,7273.613660
min,1001.000000,-14.219840,-176.629620,0.000000,0.000000,0.000000,0.000194,0.112768,0.134153,0.011449,1.000000,5.770000e+02,5.770000e+02,90.000000
25%,27052.750000,35.427243,-97.255342,1253.750000,8.200000,0.000000,3.050420,59.434074,19.513217,61.286177,19.000000,3.247500e+04,3.287925e+04,580.000000
50%,49780.500000,39.502970,-88.224820,4099.000000,29.000000,0.000000,11.541912,125.957938,35.131017,207.328858,74.000000,1.128665e+05,1.140365e+05,1880.000000
75%,72210.250000,42.110000,-80.289333,16113.250000,238.825000,0.000000,30.046597,209.476362,55.231087,471.857289,334.000000,5.052290e+05,5.092270e+05,7560.000000
max,99929.000000,71.274340,144.876370,130352.000000,68424.500000,1.000000,10767.713382,6127.454722,5101.697518,1336.551268,7263.000000,1.522467e+07,1.527798e+07,61920.000000


In [44]:
zip_code_demographics_df_raw.describe(include = 'object')

,city,state_id,state_name,county_name
count,33704,33704,33704,33704
unique,17551,51,51,1800
top,Houston,TX,Texas,Washington
freq,106,1990,1990,401


## Cleaning Zip Code Demographics Data

In [45]:
zip_code_demographics_df_raw.duplicated().sum()

np.int64(0)

In [46]:
# Keep only the necessary columns
zip_code_demographics_df = zip_code_demographics_df_raw [['zip', 'population', 'density', 'dist_highway', 'dist2_large_airport', 'dist_to_shore', 'number_of_business', 'adjusted_gross_income']]

In [47]:
total_missing = zip_code_demographics_df.isna().sum() * 100 / len(zip_code_demographics_df)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
zip,0.000000
population,0.000000
density,0.000000
dist_highway,0.000000
dist2_large_airport,0.000000
dist_to_shore,0.000000
number_of_business,0.237361
adjusted_gross_income,0.000000


In [50]:
# Drop nulls
zip_code_demographics_df = zip_code_demographics_df.dropna()
# Reset and drop the index
zip_code_demographics_df = zip_code_demographics_df.reset_index(drop=True)

In [52]:
total_missing = zip_code_demographics_df.isna().sum() * 100 / len(zip_code_demographics_df)
print('Percentage Missing Value %')
total_missing

Percentage Missing Value %


,0
zip,0.0
population,0.0
density,0.0
dist_highway,0.0
dist2_large_airport,0.0
dist_to_shore,0.0
number_of_business,0.0
adjusted_gross_income,0.0


In [53]:
# cast date types
zip_code_demographics_df = zip_code_demographics_df.astype({'zip': 'int', 'population': 'float', 'density': 'float', 'dist_highway': 'float', 'dist2_large_airport': 'float', 'dist_to_shore': 'float', 'number_of_business': 'float', 'adjusted_gross_income': 'float'})

In [54]:
zip_code_demographics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33624 entries, 0 to 33623
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   zip                    33624 non-null  int64  
 1   population             33624 non-null  float64
 2   density                33624 non-null  float64
 3   dist_highway           33624 non-null  float64
 4   dist2_large_airport    33624 non-null  float64
 5   dist_to_shore          33624 non-null  float64
 6   number_of_business     33624 non-null  float64
 7   adjusted_gross_income  33624 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 2.1 MB


In [55]:
zip_code_demographics_df.rename(columns={'zip': 'zip_code'}, inplace=True)

## Merge real_estate_df with zip_code_demographics_df to add the zip code related features

In [56]:
real_estate_features_df = pd.merge(real_estate_df, zip_code_demographics_df, on='zip_code', how='inner')
real_estate_features_df.head(10)

,price,bed,bath,acre_lot,city,zip_code,house_size,price_per_sqft,county,state,population,density,dist_highway,dist2_large_airport,dist_to_shore,number_of_business,adjusted_gross_income
0,215000.0,3,2,0.19,Chicopee,1020,1828.0,117.614880,Hampden County,MA,30704.0,951.4,0.925140,106.769095,105.789421,550.0,828125.0
1,389900.0,3,1,24.00,Northampton,1062,1559.0,250.096216,Hampshire County,MA,9835.0,217.6,3.630387,122.235452,124.761768,191.0,639133.0
2,269900.0,2,1,0.46,South Hadley,1075,1312.0,205.716463,Hampshire County,MA,18051.0,393.3,2.910859,112.181953,115.181126,297.0,697896.0
3,314900.0,5,2,0.28,Chicopee,1013,2219.0,141.910771,Hampden County,MA,22709.0,1549.1,0.062286,109.114246,104.576258,317.0,516431.0
4,244999.0,4,1,0.23,Springfield,1104,1285.0,190.660700,Hampden County,MA,23226.0,1714.0,0.645911,104.785728,101.063511,528.0,453300.0
5,199000.0,2,2,0.85,Deerfield,1373,1664.0,119.591346,Franklin County,MA,5076.0,85.0,2.294487,126.845728,139.556753,139.0,224315.0
6,279000.0,6,3,0.13,Chicopee,1020,3348.0,83.333333,Hampden County,MA,30704.0,951.4,0.925140,106.769095,105.789421,550.0,828125.0
7,589000.0,4,3,1.60,Ludlow,1056,2964.0,198.717949,Hampden County,MA,21050.0,298.9,2.608102,99.921771,105.057502,412.0,747067.0
8,399900.0,3,3,0.35,Easthampton,1027,1380.0,289.782609,Hampshire County,MA,17708.0,168.5,9.873599,125.949999,124.367513,372.0,671249.0
9,359000.0,3,2,1.03,Westfield,1085,1670.0,214.970060,Hampden County,MA,41715.0,269.7,2.075971,118.450463,101.617250,845.0,1371042.0


In [57]:
real_estate_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451719 entries, 0 to 451718
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   price                  451719 non-null  float64
 1   bed                    451719 non-null  int64  
 2   bath                   451719 non-null  int64  
 3   acre_lot               451719 non-null  float64
 4   city                   451719 non-null  object 
 5   zip_code               451719 non-null  int64  
 6   house_size             451719 non-null  float64
 7   price_per_sqft         451719 non-null  float64
 8   county                 451719 non-null  object 
 9   state                  451719 non-null  object 
 10  population             451719 non-null  float64
 11  density                451719 non-null  float64
 12  dist_highway           451719 non-null  float64
 13  dist2_large_airport    451719 non-null  float64
 14  dist_to_shore          451719 non-nu

In [58]:
real_estate_features_df.describe()

,price,bed,bath,acre_lot,zip_code,house_size,price_per_sqft,population,density,dist_highway,dist2_large_airport,dist_to_shore,number_of_business,adjusted_gross_income
count,4.517190e+05,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,451719.000000,4.517190e+05
mean,5.518166e+05,3.284365,2.428187,11.148105,59483.667176,1943.094021,285.082448,33545.607781,1265.905413,6.976981,76.397552,176.731627,691.471818,1.374152e+06
std,7.508133e+05,1.315466,1.195087,774.598407,31654.732566,1763.175229,240.130182,19403.167687,1784.271593,14.525969,101.157852,235.796885,513.583607,1.163807e+06
min,1.000000e+00,1.000000,1.000000,0.000000,1001.000000,100.000000,0.000282,0.000000,0.000000,0.000283,0.424420,0.011449,1.000000,2.925000e+03
25%,2.499500e+05,3.000000,2.000000,0.130000,31320.000000,1303.000000,150.142993,19787.000000,244.800000,1.365326,19.630244,18.578272,319.000000,6.152530e+05
50%,3.950000e+05,3.000000,2.000000,0.190000,70601.000000,1714.000000,214.863498,31480.000000,811.700000,2.912591,36.311981,65.523818,585.000000,1.108688e+06
75%,6.250000e+05,4.000000,3.000000,0.320000,91106.000000,2300.000000,333.236996,44608.000000,1707.000000,6.403780,90.491868,270.690594,943.000000,1.785317e+06
max,8.000000e+07,444.000000,222.000000,100000.000000,99402.000000,927828.000000,15015.015015,130352.000000,58289.600000,299.940968,994.951376,1330.442647,7263.000000,1.522467e+07


In [59]:
real_estate_features_df.describe(include = 'object')

,city,county,state
count,451719,451719,451719
unique,8038,1243,49
top,Houston,Los Angeles County,CA
freq,10570,20835,93286
